In [1]:
from PIL import Image
import requests
import torch
from huggingface_hub import hf_hub_download
import pandas as pd
from open_flamingo import create_model_and_transforms

model, image_processor, tokenizer = create_model_and_transforms(
    clip_vision_encoder_path="ViT-L-14",
    clip_vision_encoder_pretrained="openai",
    lang_encoder_path="anas-awadalla/mpt-1b-redpajama-200b",
    tokenizer_path="anas-awadalla/mpt-1b-redpajama-200b",
    cross_attn_every_n_layers=1
)


/data/home/jielinq/miniconda3/envs/openflamingo/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using pad_token, but it is not set yet.
/data/home/jielinq/.cache/huggingface/modules/transformers_modules/anas-awadalla/mpt-1b-redpajama-200b/bfa38d4f431e091fe599d7b4cdb62972532f3c7c/attention.py:289: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn(


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.
Flamingo model initialized with 1046992944 trainable parameters


In [2]:
import os
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from pycocoevalcap.cider import cider
from pycocoevalcap.spice import spice
from bert_score import score
import pandas as pd
import numpy as np

In [3]:
checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-3B-vitl-mpt1b", "checkpoint.pt")
model.load_state_dict(torch.load(checkpoint_path), strict=False)

_IncompatibleKeys(missing_keys=['vision_encoder.class_embedding', 'vision_encoder.positional_embedding', 'vision_encoder.proj', 'vision_encoder.conv1.weight', 'vision_encoder.ln_pre.weight', 'vision_encoder.ln_pre.bias', 'vision_encoder.transformer.resblocks.0.ln_1.weight', 'vision_encoder.transformer.resblocks.0.ln_1.bias', 'vision_encoder.transformer.resblocks.0.attn.in_proj_weight', 'vision_encoder.transformer.resblocks.0.attn.in_proj_bias', 'vision_encoder.transformer.resblocks.0.attn.out_proj.weight', 'vision_encoder.transformer.resblocks.0.attn.out_proj.bias', 'vision_encoder.transformer.resblocks.0.ln_2.weight', 'vision_encoder.transformer.resblocks.0.ln_2.bias', 'vision_encoder.transformer.resblocks.0.mlp.c_fc.weight', 'vision_encoder.transformer.resblocks.0.mlp.c_fc.bias', 'vision_encoder.transformer.resblocks.0.mlp.c_proj.weight', 'vision_encoder.transformer.resblocks.0.mlp.c_proj.bias', 'vision_encoder.transformer.resblocks.1.ln_1.weight', 'vision_encoder.transformer.resbloc

In [10]:
from PIL import Image
import requests
import torch

"""
Step 1: Load images
"""
demo_image_one = Image.open(
    requests.get(
        "http://images.cocodataset.org/val2017/000000039769.jpg", stream=True
    ).raw
)

demo_image_two = Image.open(
    requests.get(
        "http://images.cocodataset.org/test-stuff2017/000000028137.jpg",
        stream=True
    ).raw
)

query_image = Image.open(
    requests.get(
        "http://images.cocodataset.org/test-stuff2017/000000028352.jpg", 
        stream=True
    ).raw
)


"""
Step 2: Preprocessing images
Details: For OpenFlamingo, we expect the image to be a torch tensor of shape 
 batch_size x num_media x num_frames x channels x height x width. 
 In this case batch_size = 1, num_media = 3, num_frames = 1,
 channels = 3, height = 224, width = 224.
"""
vision_x = [image_processor(demo_image_one).unsqueeze(0), image_processor(demo_image_two).unsqueeze(0), image_processor(query_image).unsqueeze(0)]
vision_x = torch.cat(vision_x, dim=0)
vision_x = vision_x.unsqueeze(1).unsqueeze(0)

"""
Step 3: Preprocessing text
Details: In the text we expect an <image> special token to indicate where an image is.
 We also expect an <|endofchunk|> special token to indicate the end of the text 
 portion associated with an image.
"""
tokenizer.padding_side = "left" # For generation padding tokens should be on the left
lang_x = tokenizer(
    ["<image>An image of two cats.<|endofchunk|><image>An image of a bathroom sink.<|endofchunk|><image>An image of"],
    return_tensors="pt",
)


"""
Step 4: Generate text
"""
generated_text = model.generate(
    vision_x=vision_x,
    lang_x=lang_x["input_ids"],
    attention_mask=lang_x["attention_mask"],
    max_new_tokens=20,
    num_beams=3,
)

print("Generated text: ", tokenizer.decode(generated_text[0]))

Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


Generated text:  <image>An image of two cats.<|endofchunk|><image>An image of a bathroom sink.<|endofchunk|><image>An image of a buffet table.<|endofchunk|>


In [14]:
tokenizer.decode(generated_text[0])[98:-15]

'An image of a buffet table'

In [4]:
def get_result(image_path, question):
    
    

    """
    Step 1: Load images
    """
    demo_image_one = Image.open(
        requests.get(
            "http://images.cocodataset.org/val2017/000000039769.jpg", stream=True
        ).raw
    )

    demo_image_two = Image.open(
        requests.get(
            "http://images.cocodataset.org/test-stuff2017/000000028137.jpg",
            stream=True
        ).raw
    )

    query_image = Image.open(image_path)


    """
    Step 2: Preprocessing images
    Details: For OpenFlamingo, we expect the image to be a torch tensor of shape 
     batch_size x num_media x num_frames x channels x height x width. 
     In this case batch_size = 1, num_media = 3, num_frames = 1,
     channels = 3, height = 224, width = 224.
    """
    vision_x = [image_processor(demo_image_one).unsqueeze(0), image_processor(demo_image_two).unsqueeze(0), image_processor(query_image).unsqueeze(0)]
    vision_x = torch.cat(vision_x, dim=0)
    vision_x = vision_x.unsqueeze(1).unsqueeze(0)

    """
    Step 3: Preprocessing text
    Details: In the text we expect an <image> special token to indicate where an image is.
     We also expect an <|endofchunk|> special token to indicate the end of the text 
     portion associated with an image.
    """
    tokenizer.padding_side = "left" # For generation padding tokens should be on the left
    lang_x = tokenizer(
        ["<image>An image of two cats.<|endofchunk|><image>An image of a bathroom sink.<|endofchunk|><image>%s"%question],
        return_tensors="pt",
    )


    """
    Step 4: Generate text
    """
    generated_text = model.generate(
        vision_x=vision_x,
        lang_x=lang_x["input_ids"],
        attention_mask=lang_x["attention_mask"],
        max_new_tokens=20,
        num_beams=3,
    )

    #print("Generated text: ", tokenizer.decode(generated_text[0]))
    
    answer = tokenizer.decode(generated_text[0])[98:-15]
    
    return answer

In [5]:
def split_words(sentence):
    return sentence.split(' ')

def Rouge(GT_caption, generated_caption): # returns list of all rouge_n needed in (precision,recall,fmeasure) for each
    res = []
    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
    output = scorer.score(GT_caption, generated_caption)
    for r in output:
        score = output[r]
        res.append((score.precision, score.recall, score.fmeasure))
    return res[0][2], res[1][2], res[2][2]

def Bleu_4(GT_caption, generated_caption):
    GT_words = split_words(GT_caption)
    gen_words = split_words(generated_caption)
    weights = (1./4., 1./4., 1./4., 1./4.)
    return corpus_bleu([[GT_words]], [gen_words], weights)

def METEOR(GT_caption, generated_caption):
    GT_words = split_words(GT_caption)
    gen_words = split_words(generated_caption)
    return meteor_score([GT_words], gen_words)

def CIDEr(GT_caption, generated_caption):
    scorer = cider.Cider()
    return scorer.compute_score({0:[GT_caption]}, {0:[generated_caption]})[0]

def SPICE(GT_caption, generated_caption):
    scorer = spice.Spice()
    return scorer.compute_score({0:[GT_caption]}, {0:[generated_caption]})[0]

def BertScore(GT_caption, generated_caption):
    P, R, F1 = score([generated_caption], [GT_caption], lang="en", verbose=True)
    #return P.tolist()[0], R.tolist()[0], F1.tolist()[0]
    return F1.tolist()[0]

def acc_full(GT_caption, generated_caption):
    GT_caption = GT_caption.lower()
    generated_caption = generated_caption.lower()
    return int(GT_caption in generated_caption)

def acc_part(GT_caption, generated_caption):
    GT_words = split_words(GT_caption.lower())
    generated_caption = generated_caption.lower()
    
    in_generated = 0
    for word in GT_words:
        if word in generated_caption:
            in_generated += 1
            
    return in_generated / len(GT_words)

In [6]:
def answer_evaluation(GT_caption, generated_caption):
    rouge1, rouge2, rougeL = Rouge(GT_caption, generated_caption)
    bleu4 = Bleu_4(GT_caption, generated_caption)
    meteor = METEOR(GT_caption, generated_caption)
    cider= CIDEr(GT_caption, generated_caption)
    spice = SPICE(GT_caption, generated_caption)
    bertscore = BertScore(GT_caption, generated_caption)
    
    return rouge1, rouge2, rougeL, bleu4, meteor, cider, spice, bertscore

In [7]:
old_tsv = pd.read_csv("landmark_all.tsv", sep='\t', encoding='utf-8')

df_results = pd.DataFrame(columns=['GT_answer', 'generated_answer','rouge1', 'rouge2', 'rougeL', 'bleu4', 'meteor', 'cider', 'spice', 'bertscore'])

In [15]:
for i in range(0,2001,50):
    
    print(i)
    GT_answer = old_tsv.loc[i,'answer']
    generated_answer = get_result(old_tsv.loc[i,'image_path'], old_tsv.loc[i,'question'])
    rouge1_res, rouge2_res, rougeL_res, bleu4_res, meteor_res, cider_res, spice_res, bertscore_res = answer_evaluation(GT_answer, generated_answer)
    
    df_results = df_results.append({'GT_answer':GT_answer, 'generated_answer': generated_answer, 'rouge1':rouge1_res, 'rouge2':rouge2_res,'rougeL':rougeL_res, \
                                        'bleu4':bleu4_res, 'meteor':meteor_res, \
                                        'cider':cider_res, 'spice':spice_res, 'bertscore':bertscore_res}, ignore_index=True)
    
df_results.to_csv('openflamingo_landmark.tsv', sep='\t', encoding='utf-8', index= False)

0


Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.
/data/home/jielinq/miniconda3/envs/openflamingo/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/data/home/jielinq/miniconda3/envs/openflamingo/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
Parsing reference captions
Parsing test captions


SPICE evaluation took: 717.8 ms


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


...

100%|██████████| 1/1 [00:00<00:00, 66.75it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 566.34it/s]

done in 0.02 seconds, 48.10 sentences/sec



/tmp/ipykernel_4069795/3282533479.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({'GT_answer':GT_answer, 'generated_answer': generated_answer, 'rouge1':rouge1_res, 'rouge2':rouge2_res,'rougeL':rougeL_res, \
